# Analyze Product Sentiment

In [ ]:
import turicreate as tc

# Read product review data

In [ ]:
products = tc.SFrame('./amazon_baby.sframe')

# Explore data

## Peek the content of the data

In [ ]:
products

## Group the product by name and count the number of review

In [ ]:
products.groupby('name', operations={'count':tc.aggregate.COUNT()}).sort('count', ascending=False)

## Examine the reivews for the most-reviewed product

In [ ]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [ ]:
giraffe_reviews

In [ ]:
len(giraffe_reviews)

In [ ]:
giraffe_reviews['rating'].show()

# Building a sentiment classifier

## Build word count vectors

In [ ]:
products['word_count'] = tc.text_analytics.count_words(products['review'])

In [ ]:
products

# Define what is positive and negative sentiment

In [ ]:
products['rating'].show()

In [ ]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [ ]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [ ]:
products

In [ ]:
products['sentiment'].show()

# Train our sentiment classifier

In [ ]:
train_data, test_data = products.random_split(.8, seed=0)

In [ ]:
sentiment_model = tc.logistic_classifier.create(train_data, target='sentiment', features=['word_count'], validation_set=test_data)

# Apply the sentiment classifier to better understand the Giraffe reviews

In [ ]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [ ]:
products

In [ ]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [ ]:
giraffe_reviews

# Sort the Giraffe reviews according to predicted sentiment

In [ ]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [ ]:
giraffe_reviews

In [ ]:
giraffe_reviews.tail()

## Show the most positive reviews

In [ ]:
giraffe_reviews[0]['review']

In [ ]:
giraffe_reviews[1]['review']

# Most negative reivews

In [ ]:
giraffe_reviews[-1]['review']

In [ ]:
giraffe_reviews[-2]['review']

# Using selected words

In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [ ]:
for word in selected_words:
    products[word] = products.apply(lambda row: row["word_count"][word] if word in row["word_count"] else 0)

In [ ]:
products.head()

### Most used selected words

In [ ]:
def most_used_selected_words(selected_words, products):
    print(selected_words)
    highest_frequency = 0
    most_used_word = ""
    for word in selected_words:
        print(word)
        word_count = products[word].sum()
        print(word_count)
        if word_count > highest_frequency:
            highest_frequency = word_count
            most_used_word = word
    return most_used_word

In [ ]:
print(most_used_selected_words(selected_words, products))

### Least used selected words

In [ ]:
def least_used_selected_words(selected_words, products):
    print(selected_words)
    lowest_frequency = 0
    least_used_word = ""
    for word in selected_words:
        print(word)
        word_count = products[word].sum()
        print(word_count)
        if word_count > lowest_frequency:
            lowest_frequency = word_count
            least_used_word = word
    return least_used_word

In [ ]:
train_data, test_data = products.random_split(.8, seed=0)

In [ ]:
selected_words_model = tc.logistic_classifier.create(train_data, target='sentiment', features=selected_words, validation_set=test_data)

In [ ]:
selected_words_model.coefficients.sort("value").print_rows(num_rows=12)

In [ ]:
selected_words_model.evaluate(test_data)

In [ ]:
sentiment_model.evaluate(test_data)

In [ ]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [ ]:
diaper_champ_reviews

In [ ]:
diaper_champ_reviews.sort("predicted_sentiment", ascending=False)[0]["review"]

In [ ]:
selected_words_model.predict(diaper_champ_reviews.sort("predicted_sentiment", ascending=False), output_type = 'probability')